<a href="https://colab.research.google.com/github/racoope70/daytrading-with-ml/blob/main/multi_stock_ppo_live_inference_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get remove --purge -y cuda* libcuda* nvidia* || echo "No conflicting CUDA packages"
!apt-get autoremove -y
!apt-get clean

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'cuda-toolkit-12-4-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-7' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-8' for glob 'cuda*'
Note, selecting 'cuda-toolkit-12-5-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-2' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-3' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-4' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-5' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-6' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-8' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-9' for glob 'cuda*'
Note, selecting 'cuda-toolkit-12-6-config

In [2]:
#Protocol Buffer Fix (for TensorFlow)
!pip uninstall -y protobuf
!pip install protobuf==3.20.3

Found existing installation: protobuf 5.29.5
Uninstalling protobuf-5.29.5:
  Successfully uninstalled protobuf-5.29.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.


In [1]:
#Update Colab Environment and System Libraries
!apt-get update -y && apt-get upgrade -y


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [2]:
#Install Correct Version of CUDA for Colab GPU
!apt-get update -qq && apt-get install -y \
    libcusolver11 libcusparse11 libcurand10 libcufft10 libnppig10 libnppc10 libnppial10 \
    cuda-toolkit-12-4

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libnppig10
E: Unable to locate package libnppc10
E: Unable to locate package libnppial10


In [3]:
#Set Correct CUDA Paths
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda-12.4'
os.environ['PATH'] += ':/usr/local/cuda-12.4/bin'
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda-12.4/lib64'


In [4]:
#Install RAPIDS and NVIDIA Dependencies
!pip install --extra-index-url=https://pypi.nvidia.com \
    cuml-cu12==25.2.0 cudf-cu12==25.2.0 cupy-cuda12x dask-cuda==25.2.0 dask-cudf-cu12==25.2.0


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [5]:
#Install TensorFlow (latest GPU-compatible version)
!pip install tensorflow==2.18.0

#Install Stable Baselines3 and Trading Libraries
!pip install stable-baselines3[extra] gymnasium gym-anytrading yfinance xgboost joblib

#Install Miscellaneous Libraries
!pip install matplotlib scikit-learn pandas numba==0.61.0

#Install PyTorch with GPU Support
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
#Install TensorFlow (latest GPU-compatible version)
!pip install tensorflow==2.18.0


import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("TensorFlow GPU memory growth enabled")
    except RuntimeError as e:
        print(f"TensorFlow GPU memory config failed: {e}")


TensorFlow GPU memory growth enabled


In [7]:
# Install all required packages
!pip install stable-baselines3[extra] gymnasium gym-anytrading yfinance --quiet
!pip install stable-baselines3[extra] --quiet


In [ ]:
# === PPO Data Download with Greeks, Sentiment, Noise Filtering, and Regime Detection ===
import os, gc, time, json, pywt, joblib
import yfinance as yf
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.cluster import KMeans
from transformers import pipeline

# === Ticker List & CONFIG ===
test_mode = True  # toggle for fast testing
ticker_list = ['TSLA', 'NVDA'] if test_mode else [
    'AAPL', 'TSLA', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'BRK-B', 'JPM', 'JNJ',
    'XOM', 'V', 'PG', 'UNH', 'MA', 'HD', 'LLY', 'MRK', 'PEP', 'KO',
    'BAC', 'ABBV', 'AVGO', 'PFE', 'COST', 'CSCO', 'TMO', 'ABT', 'ACN', 'WMT',
    'MCD', 'ADBE', 'DHR', 'CRM', 'NKE', 'INTC', 'QCOM', 'NEE', 'AMD', 'TXN',
    'AMGN', 'UPS', 'LIN', 'PM', 'UNP', 'BMY', 'LOW', 'RTX', 'CVX', 'IBM',
    'GE', 'SBUX', 'ORCL'
]

CONFIG = {
    'symbols': ticker_list,
    'period': '720d',
    'interval': '1h',
    'target': 'Target',
    'strategy_name': 'ppo_walkforward_v1'
}

# === Sentiment Model ===
sentiment_model = pipeline("sentiment-analysis", model="ProsusAI/finbert")

# === Download Stock Data ===
def download_stock_data(ticker, start_date=None, end_date=None, interval="1h", max_retries=5, window_days=730):
    if start_date is None or end_date is None:
        end_date = datetime.today()
        start_date = end_date - timedelta(days=window_days)

    start_str, end_str = start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')

    for attempt in range(1, max_retries + 1):
        try:
            print(f"📥 Attempt {attempt}: Downloading {ticker}...")
            df = yf.download(ticker, start=start_str, end=end_str, interval=interval, progress=False)
            if not df.empty:
                df.reset_index(inplace=True)
                df['Symbol'] = ticker
                df['Datetime'] = pd.to_datetime(df['Datetime'] if 'Datetime' in df else df['Date'])
                return df
            raise ValueError("Empty data")
        except Exception as e:
            print(f"Error: {e}. Retrying in {attempt * 5} sec...")
            time.sleep(attempt * 5)
    print(f"Failed to download {ticker}")
    return None

# === Option Greeks Approximation ===
def mock_option_greeks(df):
    df['ImpliedVolatility'] = df['Volatility'] * 100

    def ivrank_func(x):
        if len(x) < 2:
            return np.nan
        return np.sum(x[:-1] < x[-1]) / (len(x) - 1)

    df['IVRank'] = df['ImpliedVolatility'].rolling(100).apply(ivrank_func, raw=True)
    df['Theta'] = -df['Volatility'] * 0.01
    df['Vega'] = df['Volatility'] * 0.02
    df['Gamma'] = df['Volatility'] * 0.01
    return df

# === Add Sentiment ===
def add_sentiment(df):
    try:
        sentiment_result = sentiment_model(["Stock beats earnings expectations"])[0]
        df['Sentiment'] = sentiment_result['label']
        df['SentimentScore'] = {"positive": 1, "neutral": 0, "negative": -1}.get(df['Sentiment'], 0)
    except Exception as e:
        print(f"Sentiment error: {e}")
        df['SentimentScore'] = 0
    return df

# === Feature Engineering Pipeline ===
def compute_enhanced_features(df, ticker=None):
    df = df.copy()
    df.columns = df.columns.get_level_values(0) if isinstance(df.columns, pd.MultiIndex) else df.columns
    df = df.loc[:, ~df.columns.duplicated()]

    # === Wavelet Denoising ===
    def denoise_series(series):
        coeffs = pywt.wavedec(series, 'db4', level=2)
        coeffs[1:] = [pywt.threshold(c, value=0.5 * np.std(c)) for c in coeffs[1:]]
        return pywt.waverec(coeffs, 'db4')
    df['Denoised_Close'] = denoise_series(df['Close'].values)[:len(df)]

    # === Technical Indicators ===
    df['SMA_20'] = df['Close'].rolling(20).mean()
    df['STD_20'] = df['Close'].rolling(20).std()
    df['Upper_Band'] = df['SMA_20'] + 2 * df['STD_20']
    df['Lower_Band'] = df['SMA_20'] - 2 * df['STD_20']
    df['Lowest_Low'] = df['Low'].rolling(14).min()
    df['Highest_High'] = df['High'].rolling(14).max()
    denom = (df['Highest_High'] - df['Lowest_Low']).replace(0, np.nan)
    df['Stoch'] = ((df['Close'] - df['Lowest_Low']) / denom) * 100
    df['ROC'] = df['Close'].pct_change(10)
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).cumsum()
    df['CCI'] = ((df['High'] + df['Low'] + df['Close']) / 3 - df['Close'].rolling(20).mean()) / (0.015 * df['Close'].rolling(20).std())
    df['EMA_10'] = df['Close'].ewm(span=10).mean()
    df['EMA_50'] = df['Close'].ewm(span=50).mean()
    df['EMA_200'] = df['Close'].ewm(span=200).mean()
    df['MACD_Line'] = df['Close'].ewm(span=12).mean() - df['Close'].ewm(span=26).mean()
    df['MACD_Signal'] = df['MACD_Line'].ewm(span=9).mean()

    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / (avg_loss + 1e-6)
    df['RSI'] = 100 - (100 / (1 + rs))

    # === Volatility + ATR ===
    tr = pd.concat([
        df['High'] - df['Low'],
        abs(df['High'] - df['Close'].shift()),
        abs(df['Low'] - df['Close'].shift())
    ], axis=1).max(axis=1)
    df['ATR'] = tr.rolling(14).mean()
    df['Volatility'] = df['Close'].pct_change().rolling(20).std()

    # === Velocity Features ===
    df['Velocity'] = df['Close'].diff()
    df['Acceleration'] = df['Velocity'].diff()
    df['Velocity_Norm'] = df['Velocity'] / df['Volatility']
    df['Acceleration_Norm'] = df['Acceleration'] / df['Velocity'].rolling(14).std()

    # === Return + Target ===
    df['Return'] = (df['Close'].shift(-10) - df['Close']) / df['Close']
    df['Target'] = np.select([df['Return'] > 0.02, df['Return'] < -0.02], [1, -1], default=0)

    # === Market Regime Detection ===
    np.random.seed(42)
    regime_features = df[['Volatility', 'Return']].dropna().copy()
    kmeans = KMeans(n_clusters=3, random_state=42)
    regime_features['Regime'] = kmeans.fit_predict(regime_features)
    df['Regime'] = np.nan
    df.loc[regime_features.index, 'Regime'] = regime_features['Regime']
    df['Regime'] = df['Regime'].ffill().bfill()

    df = mock_option_greeks(df)
    df = add_sentiment(df)

    df.dropna(inplace=True)

    if ticker:
        save_dir = "/content/drive/MyDrive/trading_data/singles/"
        os.makedirs(save_dir, exist_ok=True)
        df.to_csv(os.path.join(save_dir, f"{ticker}_features.csv"), index=False)

    return df

# === Batch Processing ===
all_dfs = []
end_date = datetime.today()
start_date = end_date - timedelta(days=729)

for ticker in CONFIG['symbols']:
    df_raw = download_stock_data(ticker, start_date, end_date, interval="1h")
    if df_raw is not None and not df_raw.empty:
        try:
            df_feat = compute_enhanced_features(df_raw, ticker=ticker)
            all_dfs.append(df_feat)
        except Exception as e:
            print(f"Feature engineering failed for {ticker}: {type(e).__name__} - {e}")
    else:
        print(f"No data for {ticker}")

# === Final Save ===
if all_dfs:
    df = pd.concat(all_dfs, ignore_index=True)
    print(f"Combined dataset shape: {df.shape}")
    df.to_csv("multi_stock_feature_engineered_dataset.csv", index=False)

    drive_path = "/content/drive/MyDrive/trading_data/"
    os.makedirs(drive_path, exist_ok=True)
    df.to_csv(os.path.join(drive_path, "multi_stock_feature_engineered_dataset.csv"), index=False)
    print(f"Saved to Google Drive at {drive_path}multi_stock_feature_engineered_dataset.csv")
else:
    print("No usable data found for any ticker.")


In [ ]:
!rm -rf /content/drive

In [ ]:
# === PPO Walkforward Strategy with Confidence-Based Reward, Whipsaw Penalty, and Extended Metrics ===
import os, gc, time, json, logging, torch
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from gym_anytrading.envs import StocksEnv
from gymnasium.spaces import Box
from transformers import pipeline  #For FinBERT sentiment
from google.colab import drive

# === Mount Google Drive ===
drive.mount('/content/drive')

# === Config ===
RESULTS_DIR = "/content/drive/MyDrive/Results_May_2025/ppo_walkforward_results"
FINAL_MODEL_DIR = os.path.join(RESULTS_DIR, "models")
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(FINAL_MODEL_DIR, exist_ok=True)

# === Global Flags ===
test_mode = True
simulate_latency = True

# === Logging ===
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# === Load Feature-Engineered Data ===
df = pd.read_csv("multi_stock_feature_engineered_dataset.csv")
df['Datetime'] = pd.to_datetime(df['Datetime'])

# Fallback if column missing (e.g., old file or testing)
if 'SentimentScore' not in df.columns:
    df['SentimentScore'] = 0


# === Load Sentiment Model ===
sentiment_model = pipeline("sentiment-analysis", model="ProsusAI/finbert")


# === PPO-Compatible Environment ===
class ContinuousTradingEnv(StocksEnv):
    def __init__(self, df, frame_bound, window_size):
        super().__init__(df=df.reset_index(drop=True), frame_bound=frame_bound, window_size=window_size)
        self.action_space = Box(low=-1.0, high=1.0, shape=(1,), dtype=np.float32)

    def step(self, action):
        try:
            discrete_action = 0 if action < -0.3 else 1 if action > 0.1 else 2
            result = super().step(discrete_action)
            obs, reward, terminated, truncated, info = result if len(result) == 5 else (*result, False)

            price_now = self.df.loc[self._current_tick, 'Denoised_Close']
            price_prev = self.df.loc[self._current_tick - 1, 'Denoised_Close']
            price_diff = price_now - price_prev
            volatility = self.df.loc[self._current_tick, 'Volatility']
            volatility_weight = np.clip(volatility / 0.01, 0.5, 2.0)
            sentiment_score = self.df.loc[self._current_tick, 'SentimentScore']

            action_val = float(action.flatten()[0])
            mean = self.df['Denoised_Close'].rolling(20).mean().iloc[self._current_tick]
            conf_score = abs(price_now - mean) / (self.df['STD_20'].iloc[self._current_tick] + 1e-6)
            confidence_bonus = 0.1 * conf_score if conf_score > 1.5 else 0

            self.last_action_idx = getattr(self, "last_action_idx", self._start_tick)
            self.last_action_val = getattr(self, "last_action_val", None)
            whipsaw_penalty = 0
            if self.last_action_val is not None:
                recent_change = abs(action_val - self.last_action_val)
                if (self._current_tick - self.last_action_idx < 5) and (recent_change > 1.0):
                    whipsaw_penalty = 0.05

            self.last_action_idx = self._current_tick
            self.last_action_val = action_val

            reward += (price_diff * discrete_action * volatility_weight + confidence_bonus)
            reward += 0.05 * sentiment_score if discrete_action in [1, -1] else 0
            reward -= whipsaw_penalty
            reward -= 0.001 * abs(action_val)
            reward -= 0.002 * int(action_val != 0)

            if 'Regime' in self.df.columns:
                regime = self.df.loc[self._current_tick, 'Regime']
                if regime == 2:
                    reward *= 0.5

            return obs, np.clip(reward, -1.0, 1.0), terminated, truncated, info
        except Exception as e:
            logging.error(f"Step error: {e}")
            return self.reset(), 0, True, True, {}

# === Window Generator ===
def get_walk_forward_windows(df, window_size=3000, step_size=500, min_len=1000):
    return [(start, start + window_size) for start in range(0, len(df) - min_len, step_size) if start + window_size < len(df)]

# === PPO Trainer ===
def walkforward_ppo(df, ticker, completed_tickers, window_size=3000, step_size=500, timesteps=800000):
    if ticker in completed_tickers:
        logging.info(f"Skipping {ticker} — already completed")
        return []

    logging.info(f"Starting {ticker}")
    results = []
    device = "cuda" if torch.cuda.is_available() else "cpu"
    windows = get_walk_forward_windows(df, window_size, step_size)

    for w_idx, (start, end) in enumerate(windows):
        window_start_time = time.time()
        gc.collect()

        df_window = df.iloc[start:end].reset_index(drop=True)
        avg_sentiment = df_window['SentimentScore'].mean()
        logging.info(f"{ticker} | Window {w_idx+1}: Avg Sentiment = {avg_sentiment:.2f}")

        if len(df_window) <= 52 or len(df_window) % 2 != 0:
            df_window = df_window.iloc[:-1]
            df_window = (df_window - df_window.mean()) / df_window.std()

        if 'EMA_200' in df_window.columns and not (df_window['EMA_50'].iloc[-1] > df_window['EMA_200'].iloc[-1]):
            logging.info(f"{ticker} | Window {w_idx+1}: Skipped (no uptrend)")
            continue

        if 'Regime' in df_window.columns and df_window['Regime'].iloc[-1] in [1, 2]:
            logging.info(f"{ticker} | Window {w_idx+1}: Skipped (unfavorable regime)")
            continue

        logging.info(f"{ticker} | Window {w_idx+1}/{len(windows)} | {len(df_window)} rows")
        frame_bound = (50, len(df_window) - 3)
        vec_env = DummyVecEnv([lambda: ContinuousTradingEnv(df=df_window, frame_bound=frame_bound, window_size=10)])
        env = VecNormalize(vec_env, norm_obs=True, norm_reward=True)

        model = PPO("MlpPolicy", env,
                    learning_rate=2.1e-5, n_steps=2048, batch_size=256,
                    n_epochs=10, gamma=0.99, gae_lambda=0.95,
                    clip_range=0.15, ent_coef=0.005,
                    device=device, verbose=0,
                    policy_kwargs=dict(net_arch=[128, 128]))

        model.learn(total_timesteps=timesteps)

        prefix = f"ppo_{ticker}_window{w_idx+1}"
        model.save(os.path.join(FINAL_MODEL_DIR, f"{prefix}.zip"))
        env.save(os.path.join(FINAL_MODEL_DIR, f"{prefix}_vecnorm.pkl"))
        json.dump({"features": df_window.columns.tolist()}, open(os.path.join(FINAL_MODEL_DIR, f"{prefix}_features.json"), "w"))

        obs = env.reset().copy()
        position, balance, portfolio, trade_log, buy_indices, sell_indices = 0, 100000, [], [], [], []
        entry_price = None

        for i in range(len(df_window)):
            action, _ = model.predict(obs, deterministic=True)
            step_result = env.step(action)
            obs, reward, terminated, truncated, info = step_result if len(step_result) == 5 else (*step_result, False)

            action_val = float(action.flatten()[0])
            price = df_window['Denoised_Close'].iloc[i]
            if simulate_latency:
                time.sleep(np.random.uniform(0.05, 0.15))

            slippage = price * 0.001
            if action_val > 0.5 and position == 0:
                qty = (0.1 * balance) / (price + slippage)
                position = qty
                entry_price = price
                balance -= qty * (price + slippage)
                trade_log.append("BUY")
                buy_indices.append(i)
            elif action_val < -0.5 and position > 0:
                balance += position * (price - slippage)
                position = 0
                entry_price = None
                trade_log.append("SELL")
                sell_indices.append(i)
            elif position > 0 and entry_price:
                if price <= entry_price * 0.95:
                    balance += position * (price - slippage)
                    position = 0
                    entry_price = None
                    trade_log.append("STOP_LOSS")
                    sell_indices.append(i)
                elif price >= entry_price * 1.10:
                    balance += position * (price - slippage)
                    position = 0
                    entry_price = None
                    trade_log.append("TAKE_PROFIT")
                    sell_indices.append(i)
                else:
                    trade_log.append("HOLD")
            else:
                trade_log.append("HOLD")

            portfolio.append(balance if position == 0 else balance + position * price)

        portfolio_series = pd.Series(portfolio)
        smoothed = portfolio_series.ewm(span=10).mean()

        final_value = portfolio_series.iloc[-1]
        hold_value = (100000 / df_window['Denoised_Close'].iloc[0]) * df_window['Denoised_Close'].iloc[-1]
        returns = portfolio_series.pct_change().fillna(0)
        sharpe = (returns.mean() / (returns.std() + 1e-6)) * np.sqrt(252)
        drawdown = ((portfolio_series.cummax() - portfolio_series) / portfolio_series.cummax()).max() * 100
        logging.info(f"{ticker} | Window {w_idx+1}: Trades executed: {trade_log.count('BUY')} BUY / {trade_log.count('SELL')} SELL")
        correct, total = 0, 0
        for j in range(1, len(trade_log)):
            if trade_log[j-1] in ['BUY', 'SELL'] and trade_log[j] in ['SELL', 'BUY']:
                prev_price = df_window['Denoised_Close'].iloc[j-1]
                curr_price = df_window['Denoised_Close'].iloc[j]
                if (trade_log[j-1] == 'BUY' and curr_price > prev_price) or \
                   (trade_log[j-1] == 'SELL' and curr_price < prev_price):
                    correct += 1
                total += 1
        accuracy = correct / total if total > 0 else 0

        loss_flags = (returns < 0).astype(int)
        max_consec_loss = (loss_flags.groupby((loss_flags != loss_flags.shift()).cumsum()).cumsum() * loss_flags).max()

        profit_hits, trade_count = 0, 0
        for j in range(1, len(portfolio)):
            if trade_log[j-1] in ['BUY', 'SELL'] and trade_log[j] in ['SELL', 'BUY']:
                trade_count += 1
                if portfolio[j] > portfolio[j-1]:
                    profit_hits += 1
        hit_ratio = profit_hits / trade_count if trade_count else 0

        conf_scores = [
            abs(df_window['Denoised_Close'].iloc[j] - df_window['Denoised_Close'].rolling(20).mean().iloc[j]) /
            (df_window['STD_20'].iloc[j] + 1e-6) for j in range(20, len(df_window))
        ]
        bucketed_confidences = pd.cut(conf_scores, bins=[0, 1, 2, 3, np.inf], labels=["Low", "Medium", "High", "Extreme"])
        confidence_counts = pd.Series(bucketed_confidences).value_counts().to_dict()

        results.append({
            "Ticker": ticker,
            "Window": f"{start}-{end}",
            "PPO_Portfolio": round(final_value, 2),
            "BuyHold": round(hold_value, 2),
            "Sharpe": round(sharpe, 3),
            "Drawdown_%": round(drawdown, 2),
            "Accuracy": round(accuracy, 3),
            "Max_Consec_Losses": int(max_consec_loss),
            "Hit_Ratio": round(hit_ratio, 3),
            "Confidence_Buckets": confidence_counts,
            "Winner": "PPO" if final_value > hold_value else "Buy & Hold"
        })

        plt.figure(figsize=(10, 4))
        plt.plot(smoothed, label="PPO Portfolio (Smoothed)")
        plt.axhline(hold_value, linestyle="--", color="red", label="Buy & Hold")
        plt.scatter(buy_indices, df_window['Denoised_Close'].iloc[buy_indices], color='green', marker='^', label='Buy')
        plt.scatter(sell_indices, df_window['Denoised_Close'].iloc[sell_indices], color='red', marker='v', label='Sell')
        plt.title(f"{ticker} PPO vs Buy & Hold")
        plt.legend(); plt.grid(True); plt.tight_layout()
        plt.savefig(os.path.join(RESULTS_DIR, f"{prefix}_performance.png"))
        plt.close()
        plot_path = os.path.join(RESULTS_DIR, f"{prefix}_performance.png")
        if os.path.exists(plot_path):
            logging.info(f"Plot saved: {plot_path}")
        else:
            logging.warning(f"Plot missing: {plot_path}")


        logging.info(f"{ticker} | Window {w_idx+1} done | ⏱️ {round(time.time() - window_start_time, 2)} sec")
        del env, model, df_window
        gc.collect()
        torch.cuda.empty_cache()

    return results

# === Run All Tickers ===
def run_all():
    summary_path = os.path.join(RESULTS_DIR, "ppo_walkforward_summary.csv")
    completed = set()
    if os.path.exists(summary_path):
        try:
            summary_df = pd.read_csv(summary_path)
            completed = set(summary_df['Ticker'].unique())
        except: pass

    tickers = ['TSLA', 'NVDA'] if test_mode else df['Symbol'].unique().tolist()
    all_results = []

    def safe_run(t):
        df_t = df[df['Symbol'] == t]
        if df_t.empty:
            print(f"Skipping {t}: no data in dataframe.")
            return []
        return walkforward_ppo(df_t, t, completed)

    with ThreadPoolExecutor(max_workers=min(len(tickers), os.cpu_count())) as executor:
        for result in executor.map(safe_run, tickers):
            if result: all_results.extend(result)
            pd.DataFrame(all_results).to_csv(summary_path, index=False)

    final = pd.DataFrame(all_results)
    if not final.empty:
        weights = final['Sharpe'].clip(lower=0.1).fillna(0.1)
        weighted_perf = np.average(final['PPO_Portfolio'], weights=weights)
        print("\nTop 5 by Sharpe:")
        print(final.sort_values("Sharpe", ascending=False).head())
        print(f"\nWeighted Portfolio Value (Sharpe-weighted): {round(weighted_perf, 2)}")

run_all()
